In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

hbar = 1
m = 1

try:
    from numpy import heaviside
    heaviside = heaviside
except:
    heaviside = lambda x,a: (x > 0) * 1

# Some does not have scipy.linalg.eigh_tridiagonal, so this takes care of that
# TODO: leave only one in the end
try:
    from scipy.linalg import eigh_tridiagonal
    get_eigh = lambda H: eigh_tridiagonal(np.diag(H), np.diag(H, k=1))
except ImportError:
    get_eigh = np.linalg.eigh
    
def V_crystal(x, w, b):
    L = x[0] + x[-1]
    n = (x - 10 * w) / (w + b)
    V = np.where((x > 10 * w) & (x < L - 10 * w + b) & ((n - np.floor(n)) * (w + b) < w), -100, 0)
    """
    # equivalent to
    N = len(x)
    V = np.zeros(N)
    for i in range(0, N):
        n = (x[i] - 10 * w) / (w + b)
        if x[i] < 10 * w or x[i] > L - 10 * w + b or (n - np.floor(n)) * (w + b) > w:
            V[i] = 0
        else:
            V[i] = -100
    """
    return V

def get_length(N_w, w, b):
    return 10 * w + N_w * w + (N_w - 1) * b + 10 * w

def get_stationary_states(dx, V):
    H = np.zeros((N, N))
    for i in range(0, N):
        H[i][i] = hbar**2 / (m * dx**2) + V[i]
    for i in range(0, N - 1):
        H[i][i+1] = -hbar**2 / (2 * m * dx**2)
        H[i+1][i] = -hbar**2 / (2 * m * dx**2)
    
    eigvals, eigvecs = get_eigh(H)
    energies, waves = eigvals, eigvecs.T
    return energies, waves

def plot_potential(x, V):
    plt.plot(x, V, color="grey", linestyle="solid", linewidth=3, label="$V(x)$")

def plot_stationary_states(x, energies, waves):
    L = x[0] + x[-1]
    for n, (energy, wave) in enumerate(zip(energies, waves)):
        color = "C%d" % (n % 10)
        plt.plot((0, L), (energy, energy), color=color, linestyle="dashed")
        plt.plot(x, energy + wave * 100, color=color, linestyle="solid", label="$E_{%d} = %.2f, \psi_{%d}$" % (n, energy, n))
    
def investigate_potential(N, N_w, w, b, n):
    L = get_length(N_w, w, b)
    dx = L / (N + 1)
    x = np.linspace(dx, N * dx, N)
    V = V_crystal(x, w, b)
    
    energies, waves = get_stationary_states(dx, V)
    
    print("Greatest deviation from orthonormality:")
    print(np.max(np.abs(waves @ waves.T - np.eye(N))))

    plt.figure(figsize=(16, 8))
    plt.title("plot")
    plt.xlabel("$x / L$")
    plt.ylabel("$V(x), E_n$")
    plot_potential(x, V)
    plot_stationary_states(x, energies[:n], waves[:n])
    plt.legend(loc="upper left")
    plt.show()

N = 400
w = 0.5
b = 0.1
n = 6
investigate_potential(N, 0, w, b, n)
investigate_potential(N, 1, w, b, n)
investigate_potential(N, 2, w, b, n)
investigate_potential(N, 5, w, b, n)

# Oppgave 2

### a)
Figuren viser en potensialbrønn med utvalgte tilhørende bølgefunksjoner. Brønnens lengde og bredde er her valgt slik at det finnes tre bølgefunksjoner innenfor brønnen. Fra figuren kommer det frem at annenhver bølgeunksjon er symmetrisk og antisymmetrisk, hvilket stemmer med teori. Man serr også at bølgefunksjonene for grunntilstand, første og andre eksiterte tilstand henholdsvis har 0, 1 og 2 nullpunkter. De oppfyller også initialbetingelsene om at de skal være kontinuerlige og at de går mot null i stor avstand fra brønnen, når potensialet er null.

Denne potensialfunksjonen kan modellere et atom, og bølgefunksjonene er da mulige tilstander elektroner kan befinne seg i. 
### TODO: Sjekk energiegenverdiene opp mot analytiske verdier.

Det er også tegnet inn en bølgefunksjon ovenfor brønnen. Denne representerer bølgefunksjonen for et ubundet elektron (fritt elektron), som har for stor kinetisk energi til å fanges av det elektriske feltet til atomkjernen. Et slikt elektron skal, langt unna atomet, ha en upåvirket, sinusoidal bølgefunksjon. I figuren er dette tilfelle langt ut til sidene. 

Her kan bølgelengden til elektronet leses av fra grafen.
### TODO Les av bølgelengden. Si litt mer om elektronet?

### b)
Med tre bundne tilstander er det plass til inntil seks elektroner innenfor potensialbrønnen. Dette fordi Pauliprinsippet utelukker muligheten for at det kan eksistere mer enn to elektroner (ett med spin opp og ett med spinn ned) per bundne tilstand. En slik potensialbrønn kan modellere atomnummer fem og seks (bor og karbon), som har fem og seks elektroner i grunntilstand. Deres totale elektronspinn finnes fra Pauliprinsippet; elektronene i karbonatomets tre elektronpar må ha motsatt spinn, slik at det samlede elektronspinnet til $C^0$ må være null. Samme argumentasjon for boratomet, som har fem elektroner, gir rom for at det femte elektronet (som med stor sannsynlighet befinner seg i den mest energirike andre eksiterte tilstanden) kan ha spinn op eller ned, slik at atomets totale elektronspinn er $S = \pm \frac{\sqrt{3}}{2} \hbar$